# Reference Dataset Curation

This notebook documents the curation of four reference datasets of `.mol` files used in this repository: `gdb13_1201`, `gdb17_200`, `coconut_55`, and `checks`. 

In [ ]:
# Import necessary packages.
from collections import defaultdict
import gzip
import numpy as np
import os
import os.path as osp
import pandas as pd
from rdkit import Chem
import shutil
import tarfile
from urllib.request import urlretrieve
import zipfile

# Set up random number generation.
rng = np.random.default_rng(137)

## Conversion between molecular representation formats

The `assembly-theory` package primarily uses the `.mol` molecular representation.
The GDB-13 and GDB-17 databases contain molecules in `.smi` format, which is for the SMILES molecular representation.
Additionally, the COCONUT database contains molecules in `.sdf` format, which is closely related to the `.mol` format.
The following functions utilize RDKit to convert between representations using the [`rdkit.Chem.rdchem.Mol`](https://www.rdkit.org/docs/source/rdkit.Chem.rdchem.html#rdkit.Chem.rdchem.Mol) class.

In [ ]:
def smiles_to_sdf(smiles_list, sdf_output_name):
    """
    Converts a list of SMILES molecule strings into a single .sdf file. 

    :param smiles_list: a list of SMILES molecule strings
    :param sdf_output_name: a string filename of the output .sdf file
    """
    writer = Chem.SDWriter(sdf_output_name)
    for smiles in smiles_list:
        writer.write(Chem.MolFromSmiles(smiles))


def mols_to_sdf(mol_list, sdf_output_name):
    """
    Writes a list of RDKit Mol objects to a single .sdf file.

    :param mol_list: a list of RDKit Mol objects
    :param sdf_output_name: a string filename of the output .sdf file
    """
    writer = Chem.SDWriter(sdf_output_name)
    for mol_object in mol_list:
        writer.write(mol_object)


def sdf_to_molfiles(sdf_filename, output_directory):
    """
    Splits all molecules in a .sdf file into individual .mol files in a specified output directory. 

    :param sdf_filename: a string filename of the input .sdf file
    :param output_directory: a string name of the output directory
    """
    # Ensure the output directory exists.
    os.makedirs(output_directory, exist_ok=True)
    
    # Load each molecule in the .sdf file as a Mol object and write it out as a numbered .mol file.
    supplier = Chem.SDMolSupplier(sdf_filename)
    num_digits = int(np.log10(len(supplier))) + 1
    for i, mol in enumerate(supplier):
        if mol is not None:
            Chem.MolToMolFile(mol, osp.join(output_directory, f"{str(i+1).zfill(num_digits)}.mol"))

## Sampling molecules from databases

Our reference datasets contain curated subsets of larger databases such as GDB-13, GDB-17, and COCONUT.
The following function controls this (possibly-random) sampling.

In [ ]:
def smiles_subset(smi_filename, num_molecules=None, random_sample=False):
    """
    Collect a fixed-size subset of SMILES strings from the given .smi file.

    :param smi_filename: a string filename of the input .smi file
    :param num_molecules: an int number of molecules to collect, or None to collect all
    :param random_sample: True iff the SMILES strings should be sampled uniformly at random
    :returns: a list containing the specified number of SMILES strings
    """
    # Read every line in the .smi file.
    with open(smi_filename, 'r') as file:
        smiles = [line.strip() for line in file]

    # If num_molecules is None, get ready to collect all molecules in the file.
    if num_molecules is None:
        num_molecules = len(smiles)

    # If random, then do sampling uniformly at random; otherwise, take the first num_molecules strings in order.
    if random_sample:
        return [str(s) for s in rng.choice(smiles, num_molecules, replace=False)]
    else:
        return smiles[:num_molecules]

## `gdb13_1201`

This dataset contains 1,201 small, organic molecular structures sampled from [GDB-13](https://zenodo.org/record/5172018), a database of enumerated chemical structures containing Carbon, Hydrogen, Nitrogen, Oxygen, Sulfur, and Chlorine that are constrained only by valence rules and quantum mechanics but may not be chemically stable or synthesizable (Blum & Reymond, 2009).
Our sample includes all 201 molecules in GDB-13 with 4&ndash;5 heavy atoms and 200 randomly sampled molecules for each number of heavy atoms from 6&ndash;10.

The following code downloads and extracts GDB-13 and obtains our desired subset.
GDB-13 is organized into `.smi` files by numbers of heavy atoms.
For example, file `4.smi` contains only molecules with four heavy atoms.
Within these individual `.smi` files, individual molecules are sorted by their molecular complexity, meaning molecules containing primarily Carbon are listed first, and molecules containing a greater variety of atoms are listed later.

In [ ]:
# Download GDB-13 in its entirety.
print("Downloading GDB-13 from Zenodo...")
gdb13_url = "https://zenodo.org/records/5172018/files/gdb13.tgz?download=1"
urlretrieve(gdb13_url, "gdb13.tgz")
print("Done downloading.")

# Extract GDB-13 from .tgz to a new directory called gdb13/.
gdb13_dir = "gdb13"
with tarfile.open("gdb13.tgz", 'r') as f_tar:
    f_tar.extractall(path=gdb13_dir)

# Collect all molecules with 4-5 heavy atoms (43 four-heavy, 158 five-heavy).
print("Collecting the gdb13_1201 subset...")
for i in [4, 5]:
    subset = smiles_subset(osp.join(gdb13_dir, f"{i}.smi"))
    smiles_to_sdf(subset, sdf_output_name=f"{i}.sdf")

# For each number of heavy atoms from 6-10, collect 200 molecules uniformly at random.
for i in range(6, 11):
    subset = smiles_subset(osp.join(gdb13_dir, f"{i}.smi"), num_molecules=200, random_sample=True)
    smiles_to_sdf(subset, sdf_output_name=f"{i}.sdf")

# Concatenate these .sdf files into one gdb13_1201.sdf file for our reference dataset.
with open("gdb13_1201.sdf", 'wb') as f_out:
    for i in range(4, 11):
        with open(f"{i}.sdf", 'rb') as f_in:
            shutil.copyfileobj(f_in, f_out)

# Split this .sdf file into individual .mol files.
sdf_to_molfiles("gdb13_1201.sdf", output_directory=osp.join("..", "data", "gdb13_1201"))

# Clean up all intermediate files.
print("Deleting intermediate files...")
os.remove("gdb13.tgz")
shutil.rmtree(gdb13_dir)
for i in range(4, 11):
    os.remove(f"{i}.sdf")
os.remove("gdb13_1201.sdf")

## `gdb17_200`

This dataset contains 200 organic molecular structures sampled from the larger [GDB-17](https://zenodo.org/record/5172018) database, which includes additional nuclei beyond GDB-13 such as the halogens Flourine and Iodine (Ruddigkeit et al., 2012).
Compared to GDB-13, these molecules are typically larger and represent more structural diversity.
Our sample includes 50 randomly sampled molecules for each number of heavy atoms from 14&ndash;17.

GDB-17 downloads as a single `.smi` file containing 50 million molecules.
Obtaining our sample of 50 molecules per number of heavy atoms requires slightly more effort than in GDB-13, where molecules were already split out by number of heavy atoms.

In [ ]:
# Download GDB-17 in its entirety.
print("Downloading GDB-17 from Zenodo...")
gdb17_url = "https://zenodo.org/records/5172018/files/GDB17.50000000.smi.gz?download=1"
urlretrieve(gdb17_url, "gdb17.gz")
print("Done downloading.")

# Extract GDB-17 from .gz to a .smi file.
with gzip.open("gdb17.gz", 'rb') as f_in:
    with open("gdb17.smi", 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

# Downsample from 50M to 100K molecules uniformly at random.
print("Collecting the gdb17_200 subset...")
sample = smiles_subset("gdb17.smi", num_molecules=100000, random_sample=True)

# Write a quick function to get numbers of heavy atoms from SMILES strings.
def heavy(smiles):
    return Chem.MolFromSmiles(smiles).GetNumHeavyAtoms()

# Create a pandas dataframe with the SMILES string and number of heavy atoms.
smiles_df = pd.DataFrame({'smiles': sample})
smiles_df['heavy'] = smiles_df.map(heavy)

# For each number of heavy atoms from 14-17, collect 50 molecules into an .sdf file.
for i in range(14, 18):
    subset = smiles_df[smiles_df['heavy'] == i]['smiles'].tolist()
    smiles_to_sdf(subset[:50], sdf_output_name=f"{i}.sdf")

# Concatenate these .sdf files into one gdb17_200.sdf file for our reference dataset.
with open("gdb17_200.sdf", 'wb') as f_out:
    for i in range(14, 18):
        with open(f"{i}.sdf", 'rb') as f_in:
            shutil.copyfileobj(f_in, f_out)

# Split this .sdf file into individual .mol files.
sdf_to_molfiles("gdb17_200.sdf", output_directory=osp.join("..", "data", "gdb17_200"))

# Clean up all intermediate files.
print("Deleting intermediate files...")
os.remove("gdb17.gz")
os.remove("gdb17.smi")
for i in range(14, 18):
    os.remove(f"{i}.sdf")
os.remove("gdb17_200.sdf")

## `coconut_55`

This dataset contains 55 natural products sampled from the COCONUT database (Sorokina et al., 2021), [accessed in late 2024](https://zenodo.org/records/13897048) prior to COCONUT 2.0 (Chandrasekhar et al., 2025).
Natural products (or secondary metabolites) are a rich source of evolved chemical complexity, often exhibiting drug-like properties.
Subsets of this database were used to benchmark recent algorithmic progress in (Seet et al., 2024).
Our sample includes 5 randomly sampled molecules for each number of heavy atoms from 15&ndash;25.

COCONUT downloads as a single `.sdf` file containing all molecules in the database.
Like GDB-17, this requires some effort to identify molecules' number of heavy atoms and sample from them.

In [ ]:
# Download COCONUT in its entirety.
print("Downloading COCONUT from Zenodo...")
coconut_url = "https://zenodo.org/records/13897048/files/coconut_complete-10-2024.sdf.zip?download=1"
urlretrieve(coconut_url, "coconut.zip")
print("Done downloading.")

# Extract COCONUT from .zip to a new directory called coconut/.
with zipfile.ZipFile("coconut.zip", 'r') as f_zip:
    f_zip.extractall(path='coconut')

# Read molecules from .sdf, filtering and sorting by number of heavy atoms.
print("Collecting the coconut_55 subset...")
mols = defaultdict(list)
for mol in Chem.SDMolSupplier(osp.join('coconut', 'coconut_complete-10-2024.sdf')):
    if mol is not None:
        num_heavy = mol.GetNumHeavyAtoms()
        if num_heavy >= 15 and num_heavy <= 25:
            mols[num_heavy].append(mol)
        
# For each number of heavy atoms from 15-25, collect 5 molecules uniformly at random.
for i in range(15, 26):
    subset = list(rng.choice(mols[i], 5, replace=False))
    mols_to_sdf(subset, sdf_output_name=f"{i}.sdf")

# Concatenate these .sdf files into one coconut_55.sdf file for our reference dataset.
with open("coconut_55.sdf", 'wb') as f_out:
    for i in range(15, 26):
        with open(f"{i}.sdf", 'rb') as f_in:
            shutil.copyfileobj(f_in, f_out)


# Split this .sdf file into individual .mol files.
sdf_to_molfiles("coconut_55.sdf", output_directory=osp.join("..", "data", "coconut_55"))

# Clean up all intermediate files.
print("Deleting intermediate files...")
os.remove("coconut.zip")
shutil.rmtree("coconut")
for i in range(15, 26):
    os.remove(f"{i}.sdf")
os.remove("coconut_55.sdf")

## `checks`

This dataset contains 15 named molecules (e.g., anthracene, aspirin, caffeine, morphine) from [KEGG COMPOUND](https://www.genome.jp/kegg/compound/) (Kanehisa, 2019; Kanehisa et al., 2023; Kanehisa & Goto, 2000).
These molecules are primarily used for rapid testing have have numbers of heavy atoms ranging from 5&ndash;21.
The code below simply documents where these molecules were downloaded from and automates the downloads.

In [ ]:
# Set the checks dataset directory and ensure it exists.
checks_dir = osp.join("..", "data", "checks")
os.makedirs(checks_dir, exist_ok=True)

# Define molecule names and download URLs.
mols = [
    ("anthracene", "https://www.kegg.jp/entry/-f+m+C14315"),
    ("aspartic", "https://www.kegg.jp/entry/-f+m+C00049"),
    ("aspirin", "https://www.kegg.jp/entry/-f+m+C01405"),
    ("benzene", "https://www.kegg.jp/entry/-f+m+C01407"),
    ("caffeine", "https://www.kegg.jp/entry/-f+m+C07481"),
    ("chrysene", "https://www.kegg.jp/entry/-f+m+C14222"),
    ("cyanidin", "https://www.kegg.jp/entry/-f+m+C05905"),
    ("ethylenethiourea", "https://www.kegg.jp/entry/-f+m+C14398"),
    ("fosfructose", "https://www.kegg.jp/entry/-f+m+C00354"),
    ("morphine", "https://www.kegg.jp/entry/-f+m+C01516"),
    ("naphthalene", "https://www.kegg.jp/entry/-f+m+C00829"),
    ("pyrazole", "https://www.kegg.jp/entry/-f+m+C00481"),
    ("pyrene", "https://www.kegg.jp/entry/-f+m+C14335"),
    ("thiouracil", "https://www.kegg.jp/entry/-f+m+C19304"),
    ("xylopyranose", "https://www.kegg.jp/entry/-f+m+C02205")
]

# Download the checks dataset.
print("Downloading checks molecules from KEGG COMPOUND...")
for mol_name, mol_url in mols:
    urlretrieve(mol_url, osp.join(checks_dir, f"{mol_name}.mol"))
print("Done downloading.")

## References

Blum, L. C., & Reymond, J.-L. (2009). 970 Million Druglike Small Molecules for Virtual Screening in the Chemical Universe Database GDB-13. *Journal of the American Chemical Society*, *131*(25), 8732–8733. https://doi.org/10.1021/ja902302h

Chandrasekhar, V., Rajan, K., Kanakam, S. R. S., Sharma, N., Weißenborn, V., Schaub, J., & Steinbeck, C. (2025). COCONUT 2.0: A comprehensive overhaul and curation of the collection of open natural products database. *Nucleic Acids Research*, *53*(D1), D634–D643. https://doi.org/10.1093/nar/gkae1063

Kanehisa, M. (2019). Toward understanding the origin and evolution of cellular organisms. *Protein Science*, *28*(11), 1947–1951. https://doi.org/10.1002/pro.3715

Kanehisa, M., Furumichi, M., Sato, Y., Kawashima, M., & Ishiguro-Watanabe, M. (2023). KEGG for taxonomy-based analysis of pathways and genomes. *Nucleic Acids Research*, *51*(D1), D587–D592. https://doi.org/10.1093/nar/gkac963

Kanehisa, M., & Goto, S. (2000). KEGG: Kyoto Encyclopedia of Genes and Genomes. *Nucleic Acids Research*, *28*(1), 27–30. https://doi.org/10.1093/nar/28.1.27

Ruddigkeit, L., Van Deursen, R., Blum, L. C., & Reymond, J.-L. (2012). Enumeration of 166 Billion Organic Small Molecules in the Chemical Universe Database GDB-17. *Journal of Chemical Information and Modeling*, *52*(11), 2864–2875. https://doi.org/10.1021/ci300415d

Seet, I., Patarroyo, K. Y., Siebert, G., Walker, S. I., & Cronin, L. (2024). *Rapid Computation of the Assembly Index of Molecular Graphs* (No. 2410.09100). arXiv. https://doi.org/10.48550/arXiv.2410.09100

Sorokina, M., Merseburger, P., Rajan, K., Yirik, M. A., & Steinbeck, C. (2021). COCONUT online: Collection of Open Natural Products database. *Journal of Cheminformatics*, *13*(1), 2. https://doi.org/10.1186/s13321-020-00478-9